In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np
import os 
from datetime import datetime


2025-10-30 02:12:16.973571: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-30 02:12:17.047067: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-30 02:12:20.785909: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
RANDOM_STATE = 42
path_DATA = "../../../../../data/spanish"

csv_path = f"{path_DATA}/dataset/oficialDatasetEAIM2026.csv"


TESTING = False

if TESTING:
    NROWS = None
else:
    NROWS = None
    


In [3]:
df = pd.read_csv(csv_path, nrows=NROWS)
display(df.head(2))


,_id,artist,genre,lyrics,composer,lyrics_word_count,title_songs_new,spotify_id,popularity,explicit_content,duration_ms,release_date,external_urls.spotify,letras_path,id_yt
0,689a8a83a437eda121bd38ce,Laura Pausini,pop,Ya no responde ni al teléfono\nPende de un hil...,Compuesta por: Federico Cavalli / Angelo Valsi...,445,Se Fue,5oQadhkuEdEhtdVn0QceyZ,71,False,240179,2024-11-15,https://open.spotify.com/track/5oQadhkuEdEhtdV...,Laura Pausini/30278,g-GBiuujmL8
1,689a8a83a437eda121bd389f,Shakira,pop,Loca\n(Loca)\nNo te ponga' bruto\n\nQue te la ...,Compuesta por: Pitbull / El Cata / Shakira / C...,360,Loca (part. El Cata),42k1KeBehAd83lrGt1okiC,76,False,183693,2010-10-19,https://open.spotify.com/track/42k1KeBehAd83lr...,Shakira/1735339,XAhTt60W7qo


In [4]:
path_train_indices = f"{path_DATA}/train_indices_spanish.csv"
path_test_indices = f"{path_DATA}/test_indices_spanish.csv"


df_train_indices = pd.read_csv(path_train_indices, nrows=NROWS)
df_test_indices = pd.read_csv(path_test_indices, nrows=NROWS)

idx_train = df_train_indices['original_index'].values
idx_test = df_test_indices['original_index'].values

display(df_train_indices.head(2))
display(df_test_indices.head(2))

df = pd.read_csv(csv_path, nrows=NROWS)
# display(df)

,_id,original_index
0,689a8a9da437eda121bd54f1,2097
1,689a8ab7a437eda121bd6def,4037


,_id,original_index
0,689a8aafa437eda121bd65ae,2988
1,689a8eaca437eda121bd84fe,6399


MoritzLaurer/deberta-v3-large-zeroshot-v2.0  
https://huggingface.co/MoritzLaurer/deberta-v3-large-zeroshot-v2.0

In [5]:
X = df['lyrics']
y = df['explicit_content']

X_train = X.loc[idx_train]
X_test = X.loc[idx_test]
y_train = y.loc[idx_train]
y_test = y.loc[idx_test]

# display(X_test.head())

hypothesis_template = "Esta letra contiene contenido {}"
classes_verbalized = ["explícito", "no explícito"]


device = 0 if torch.cuda.is_available() else -1

zeroshot_classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=device
)

Device set to use cuda:0


## Prueba con treshold 0.5

In [ ]:
best_threshold = 0.5
print("Probando con: ",best_threshold )
test_scores = []
for lyric in X_train:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explícito')]
    test_scores.append(score_explicit)

test_scores = np.array(test_scores)
test_preds = (test_scores >= best_threshold).astype(int)

# Métricas
f1_test = f1_score(y_train, test_preds)
accuracy_test = accuracy_score(y_train, test_preds)
precision_test = precision_score(y_train, test_preds)
recall_test = recall_score(y_train, test_preds)

df_results = pd.DataFrame({
    "original_index": idx_train,
    "__id": df.loc[idx_train, "_id"].values if "_id" in df.columns else None,
    "explicit_score": test_scores, 
    "prediccion": test_preds
})
df_results.to_csv(f"deberta_train_predictions.csv", index=False)


print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Probando con:  0.5
Accuracy en test: 0.66592656
Precision en test: 0.12269327
Recall en test: 0.55530474
F1 en test: 0.20098039


In [10]:
best_threshold = 0.5
print("Probando con: ",best_threshold )
test_scores = []
for lyric in X_test:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explícito')]
    test_scores.append(score_explicit)

test_scores = np.array(test_scores)
test_preds = (test_scores >= best_threshold).astype(int)

# Métricas
f1_test = f1_score(y_test, test_preds)
accuracy_test = accuracy_score(y_test, test_preds)
precision_test = precision_score(y_test, test_preds)
recall_test = recall_score(y_test, test_preds)

df_results = pd.DataFrame({
    "original_index": idx_test,
    "__id": df.loc[idx_test, "_id"].values if "_id" in df.columns else None,
    "explicit_score": test_scores, 
    "prediccion": test_preds
})
df_results.to_csv(f"deberta_test_predictions.csv", index=False)


print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Probando con:  0.5
Accuracy en test: 0.65437158
Precision en test: 0.12237094
Recall en test: 0.57657658
F1 en test: 0.20189274
